In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import shap
from flask import Flask, request
from io import StringIO

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
def predict(df_consumo, df_temp):
    df_consumo['date'] = pd.to_datetime(df_consumo['date'])
    df_consumo = df_consumo[df_consumo['date'].dt.year == 2023]
    df_consumo = df_consumo.sort_values(by='date')
    df_consumo = df_consumo[df_consumo['date'].dt.year == datetime.now().year]
    df_consumo = df_consumo.sort_values(by='date')
    df_consumo = df_consumo.rename(columns={'volume': 'volumem3'})
    df_consumo['volumem3'] = df_consumo['volumem3'].str.replace(' m3', '').astype(float)
    df_consumo = df_consumo.rename(columns={'logVolume': 'logVolumem3'})
    df_consumo['logVolumem3'] = df_consumo['logVolumem3'].str.replace(' m3', '').astype(float)
    def clean_delta_volume(x):
        if isinstance(x, list):
            return [str(item).strip() for item in x]
        elif isinstance(x, str):
            return x.split(',')
        else:
            return [str(x)]

    # Aplica la función 'clean_delta_volume' a cada elemento de la columna 'deltaVolumes'
    df_consumo['deltaVolumes'] = df_consumo['deltaVolumes'].apply(clean_delta_volume)

    # Calcula la longitud máxima de las listas en 'deltaVolumes'
    max_length = df_consumo['deltaVolumes'].apply(len).max()
    new_column_names = [f'deltaVolumes{i+1}cm3' for i in range(max_length)]
    is_array = df_consumo['deltaVolumes'].apply(lambda x: isinstance(x, list))
    df_consumo[new_column_names] = pd.DataFrame(df_consumo.loc[is_array, 'deltaVolumes'].values.tolist(), index=df_consumo.loc[is_array].index)

    df_consumo[new_column_names] = df_consumo[new_column_names].apply(lambda x: x.str.replace(' m3', '').astype(float))
    df_consumo.drop('deltaVolumes', axis=1, inplace=True)
    df_consumo = df_consumo.drop(['state', 'stateMessages', 'error', 'logDate'], axis = 1)
    # Crear un nuevo DataFrame vacío para almacenar los nuevos registros
    new_df = pd.DataFrame(columns=['date', 'volumem3', 'logVolumem3', 'device_name', 'dev_eui', 'dev_addr', 'deltaVolumes'])

    # Aplicar una función que crea los nuevos registros a partir de las columnas 'deltaVolumes'
    def create_new_rows(row):
        # Obtener los valores de las columnas que se deben conservar
        date = row['date']
        volumem3 = row['volumem3']
        logVolumem3 = row['logVolumem3']
        device_name = row['device_name']
        dev_eui = row['dev_eui']
        dev_addr = row['dev_addr']
    
        # Crear una lista vacía para almacenar los nuevos registros
        new_rows = []
    
        # Iterar sobre las columnas 'deltaVolumes' en orden inverso
        for i in range(15, 7, -1):
            # Calcular la diferencia de horas respecto al registro original
            hour_diff = 1 + i
        
            # Crear un nuevo registro con los valores correspondientes
            new_row = {
                'date': date + pd.DateOffset(hours=hour_diff),
                'volumem3': volumem3,
                'logVolumem3': logVolumem3,
                'device_name': device_name,
                'dev_eui': dev_eui,
                'dev_addr': dev_addr,
                'deltaVolumes': row[f'deltaVolumes{i}cm3']
            }
        
            # Agregar el nuevo registro a la lista
            new_rows.append(new_row)
    
        # Devolver la lista de nuevos registros
        return new_rows

    # Aplicar la función a cada fila del DataFrame original y obtener una lista de listas de nuevos registros
    new_rows_list = df_consumo.apply(create_new_rows, axis=1)

    # Convertir la lista de listas en una sola lista de nuevos registros
    new_rows_list = [item for sublist in new_rows_list for item in sublist]

    # Agregar los nuevos registros al DataFrame vacío
    new_df = new_df.append(new_rows_list, ignore_index=True)

    # Eliminar las columnas innecesarias del DataFrame original
    df_consumo = df_consumo.drop(columns=[
        'deltaVolumes1cm3', 'deltaVolumes2cm3', 'deltaVolumes3cm3',
        'deltaVolumes4cm3', 'deltaVolumes5cm3', 'deltaVolumes6cm3',
        'deltaVolumes7cm3', 'deltaVolumes8cm3', 'deltaVolumes9cm3',
        'deltaVolumes10cm3', 'deltaVolumes11cm3', 'deltaVolumes12cm3',
        'deltaVolumes13cm3', 'deltaVolumes14cm3', 'deltaVolumes15cm3'
    ])

    # Concatenar el DataFrame original con el nuevo DataFrame generado
    df_consumo = pd.concat([df_consumo, new_df])

    # Ordenar el DataFrame por fecha
    df_consumo = df_consumo.sort_values('date')

    # Restablecer los índices del DataFrame
    df_consumo = df_consumo.dropna()
    df_consumo = df_consumo.reset_index(drop=True)
    # Eliminar las columnas 'volumem3' y 'logVolumem3'
    df_consumo = df_consumo.drop(['volumem3', 'logVolumem3'], axis=1)

    # Convertir la columna 'date' al formato de fecha adecuado
    df_consumo['date'] = pd.to_datetime(df_consumo['date']).dt.date

    # Sumar los valores de 'deltaVolumes' agrupados por 'device_name' y por día
    df_con_sum = df_consumo.groupby(['date', 'device_name', 'dev_eui', 'dev_addr'])['deltaVolumes'].sum().reset_index()
    df_con_sum = df_con_sum[['date', 'deltaVolumes']]

    df_sum_date = df_con_sum.groupby('date', as_index=False).sum()
    df_temp = df_temp.rename(columns={'fecha': 'date'})
    df_temp = df_temp.drop(['provincia', 'indicativo', 'nombre', 'altitud'], axis = 1)
    patron = r'[a-zA-Z]+'

    df_temp.replace(to_replace=patron, value=np.nan, regex=True, inplace=True)
    def convert_to_hour(value):
        try:
            return int(value.split(':', 1)[0])
        except AttributeError:
            return value

    columns_to_convert = ['horaracha', 'horatmin', 'horatmax']

    for column in columns_to_convert:
        df_temp[column] = df_temp[column].apply(convert_to_hour)
        
    columnas_a_convertir = ['tmed', 'prec', 'tmin', 'tmax', 'dir',
           'velmedia', 'racha', 'sol', 'presMax', 'presMin']

    df_temp[columnas_a_convertir] = df_temp[columnas_a_convertir].astype(str).apply(lambda x: x.str.replace(',', '.')).astype(float)
    aux_temp = df_temp[['tmed', 'prec', 'tmin', 'horatmin', 'tmax', 'horatmax', 'dir',
       'velmedia', 'racha', 'horaracha', 'sol', 'presMax', 'horaPresMax',
       'presMin', 'horaPresMin']]
    # Lista de columnas con NaN en el DataFrame original
    cols_with_nan = aux_temp.columns[aux_temp.isnull().any()].tolist()

    # Iterar por cada columna con NaN para predecir sus valores
    for col in cols_with_nan:
        # Separamos los registros con valores conocidos y desconocidos para esta columna
        df_known = aux_temp.dropna(subset=[col])
        df_unknown = aux_temp[aux_temp[col].isnull()]

        # Creamos el modelo de regresión lineal
        regression_model = LinearRegression()

        # Ajustamos el modelo con los datos conocidos para esta columna
        X = df_known.drop(columns=cols_with_nan).values
        y = df_known[col].values
        regression_model.fit(X, y)

        # Hacemos las predicciones para los valores desconocidos en esta columna
        X_unknown = df_unknown.drop(columns=cols_with_nan).values
        predicted_values = regression_model.predict(X_unknown)

        # Convertimos los valores predichos a enteros sin decimales
        predicted_values = predicted_values.astype(int)

        # Rellenamos los valores faltantes en el DataFrame original para esta columna
        aux_temp.loc[aux_temp[col].isnull(), col] = predicted_values
    df_tiemp = pd.concat([df_temp['date'], aux_temp], axis=1)
    df_sum_date['date'] = pd.to_datetime(df_sum_date['date'])
    df_tiemp['date'] = pd.to_datetime(df_tiemp['date'])
    df_merged = pd.merge(df_sum_date, df_tiemp, on='date', how='inner')
    df_merged['NumMes'] = df_merged['date'].dt.month
    # Crea una columna para el día de la semana (1-7, donde 1 es Lunes y 7 es Domingo)
    df_merged['DiaSem'] = df_merged['date'].dt.dayofweek + 1
    df_merged = df_merged[['NumMes', 'DiaSem'] + [col for col in df_merged.columns if col not in ['NumMes', 'DiaMes']]]
    df_merged = df_merged.drop(columns=['date'])
    df_merged['previous_deltaVolumes'] = df_merged['deltaVolumes'].shift(1)
    df_merged.dropna(inplace=True)
    column_names = df_merged.columns.tolist()

    if "deltaVolumes" in column_names:
        column_names.remove("deltaVolumes")

    column_names.append("deltaVolumes")

    df_merged = df_merged[column_names]
    df_merged['deltaVolumes'] = df_merged['deltaVolumes'].astype(float)
    df_merged['horaPresMax'] = df_merged['horaPresMax'].astype(float)
    df_merged['horaPresMin'] = df_merged['horaPresMin'].astype(float)
    # Separar las características y la variable objetivo
    X = df_merged[['NumMes', 'tmed', 'tmin', 'tmax',
       'horatmax', 'dir', 'velmedia', 'racha', 'sol', 'presMax',
       'horaPresMax', 'presMin', 'previous_deltaVolumes']]

    y = df_merged['deltaVolumes']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    df_Rand = RandomForestRegressor(n_estimators=50, max_depth=20, random_state=10)
    df_Rand.fit(X_train, y_train)
    val_Rand_predictions = df_Rand.predict(X_test)
    
    explainer = shap.Explainer(df_Rand, X_train)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test)
    shap.initjs()
    shap.force_plot(explainer.expected_value, shap_values, X_test)
    shap.force_plot(explainer.expected_value, shap_values[0, :], X.iloc[0, :])
    
    return val_Rand_predictions

In [ ]:
def lstm(df_consumo, df_consumo_hist):
    df_consumo 
    df_consumo['date'] = pd.to_datetime(df_consumo['date'])
    df_consumo = df_consumo[df_consumo['date'].dt.year == 2023]
    df_consumo = df_consumo.sort_values(by='date')
    df_consumo = df_consumo[df_consumo['date'].dt.year == datetime.now().year]
    df_consumo = df_consumo.sort_values(by='date')
    df_consumo = df_consumo.rename(columns={'volume': 'volumem3'})
    df_consumo['volumem3'] = df_consumo['volumem3'].str.replace(' m3', '').astype(float)
    df_consumo = df_consumo.rename(columns={'logVolume': 'logVolumem3'})
    df_consumo['logVolumem3'] = df_consumo['logVolumem3'].str.replace(' m3', '').astype(float)
    def clean_delta_volume(x):
        if isinstance(x, list):
            return [str(item).strip() for item in x]
        elif isinstance(x, str):
            return x.split(',')
        else:
            return [str(x)]

    # Aplica la función 'clean_delta_volume' a cada elemento de la columna 'deltaVolumes'
    df_consumo['deltaVolumes'] = df_consumo['deltaVolumes'].apply(clean_delta_volume)

    # Calcula la longitud máxima de las listas en 'deltaVolumes'
    max_length = df_consumo['deltaVolumes'].apply(len).max()
    new_column_names = [f'deltaVolumes{i+1}cm3' for i in range(max_length)]
    is_array = df_consumo['deltaVolumes'].apply(lambda x: isinstance(x, list))
    df_consumo[new_column_names] = pd.DataFrame(df_consumo.loc[is_array, 'deltaVolumes'].values.tolist(), index=df_consumo.loc[is_array].index)

    df_consumo[new_column_names] = df_consumo[new_column_names].apply(lambda x: x.str.replace(' m3', '').astype(float))
    df_consumo.drop('deltaVolumes', axis=1, inplace=True)
    df_consumo = df_consumo.drop(['state', 'stateMessages', 'error', 'logDate'], axis = 1)
    # Crear un nuevo DataFrame vacío para almacenar los nuevos registros
    new_df = pd.DataFrame(columns=['date', 'volumem3', 'logVolumem3', 'device_name', 'dev_eui', 'dev_addr', 'deltaVolumes'])

    # Aplicar una función que crea los nuevos registros a partir de las columnas 'deltaVolumes'
    def create_new_rows(row):
        # Obtener los valores de las columnas que se deben conservar
        date = row['date']
        volumem3 = row['volumem3']
        logVolumem3 = row['logVolumem3']
        device_name = row['device_name']
        dev_eui = row['dev_eui']
        dev_addr = row['dev_addr']
    
        # Crear una lista vacía para almacenar los nuevos registros
        new_rows = []
    
        # Iterar sobre las columnas 'deltaVolumes' en orden inverso
        for i in range(15, 7, -1):
            # Calcular la diferencia de horas respecto al registro original
            hour_diff = 1 + i
        
            # Crear un nuevo registro con los valores correspondientes
            new_row = {
                'date': date + pd.DateOffset(hours=hour_diff),
                'volumem3': volumem3,
                'logVolumem3': logVolumem3,
                'device_name': device_name,
                'dev_eui': dev_eui,
                'dev_addr': dev_addr,
                'deltaVolumes': row[f'deltaVolumes{i}cm3']
            }
        
            # Agregar el nuevo registro a la lista
            new_rows.append(new_row)
    
        # Devolver la lista de nuevos registros
        return new_rows

    # Aplicar la función a cada fila del DataFrame original y obtener una lista de listas de nuevos registros
    new_rows_list = df_consumo.apply(create_new_rows, axis=1)

    # Convertir la lista de listas en una sola lista de nuevos registros
    new_rows_list = [item for sublist in new_rows_list for item in sublist]

    # Agregar los nuevos registros al DataFrame vacío
    new_df = new_df.append(new_rows_list, ignore_index=True)

    # Eliminar las columnas innecesarias del DataFrame original
    df_consumo = df_consumo.drop(columns=[
        'deltaVolumes1cm3', 'deltaVolumes2cm3', 'deltaVolumes3cm3',
        'deltaVolumes4cm3', 'deltaVolumes5cm3', 'deltaVolumes6cm3',
        'deltaVolumes7cm3', 'deltaVolumes8cm3', 'deltaVolumes9cm3',
        'deltaVolumes10cm3', 'deltaVolumes11cm3', 'deltaVolumes12cm3',
        'deltaVolumes13cm3', 'deltaVolumes14cm3', 'deltaVolumes15cm3'
    ])

    # Concatenar el DataFrame original con el nuevo DataFrame generado
    df_consumo = pd.concat([df_consumo, new_df])

    # Ordenar el DataFrame por fecha
    df_consumo = df_consumo.sort_values('date')

    # Restablecer los índices del DataFrame
    df_consumo = df_consumo.dropna()
    df_consumo = df_consumo.reset_index(drop=True)
    # Eliminar las columnas 'volumem3' y 'logVolumem3'
    df_consumo = df_consumo.drop(['volumem3', 'logVolumem3'], axis=1)

    # Convertir la columna 'date' al formato de fecha adecuado
    df_consumo['date'] = pd.to_datetime(df_consumo['date']).dt.date

    # Sumar los valores de 'deltaVolumes' agrupados por 'device_name' y por día
    df_con_sum = df_consumo.groupby(['date', 'device_name', 'dev_eui', 'dev_addr'])['deltaVolumes'].sum().reset_index()
    df_con_sum = df_con_sum[['date', 'deltaVolumes']]

    df_sum_date = df_con_sum.groupby('date', as_index=False).sum()

    df_sum_date = df_sum_date.rename(columns={'date': 'Fecha', 'deltaVolumes': 'Valor'})

    df_sum_date['Fecha'] = pd.to_datetime(df_sum_date['Fecha'])
    df_consumo_hist = df_consumo_hist.drop(columns=['Unidad'])
    df_consumo_hist['Fecha'] = pd.to_datetime(df_consumo_hist['Fecha']).dt.date
    df_consumo_hist['Fecha'] = pd.to_datetime(df_consumo_hist['Fecha'])
    df_consumo_hist['Valor'] = df_consumo_hist['Valor'].astype(float)
    df_datos = pd.concat([df_consumo_hist, df_sum_date], ignore_index=True)
    scaler = MinMaxScaler()
    df_datos['Valor'] = scaler.fit_transform(df_datos[['Valor']])
    df_datos = df_datos.sort_values(by="Fecha")
    df_datos = df_datos.reset_index(drop=True)
    # Función para crear secuencias de datos
    def crear_secuencias(data, pasos):
        secuencias_x, secuencias_y = [], []
        for i in range(len(data) - pasos):
            secuencias_x.append(data[i:i+pasos])
            secuencias_y.append(data[i+pasos])
        return np.array(secuencias_x), np.array(secuencias_y)

    # Definir el número de pasos de tiempo (cuántos valores pasados se usarán para predecir el futuro)
    num_pasos = 10
    X, y = crear_secuencias(df_datos['Valor'].values, num_pasos)

    # Dividir los datos en conjuntos de entrenamiento y prueba
    porcentaje_entrenamiento = 0.8
    indice_entrenamiento = int(porcentaje_entrenamiento * len(X))

    X_entrenamiento, X_prueba = X[:indice_entrenamiento], X[indice_entrenamiento:]
    y_entrenamiento, y_prueba = y[:indice_entrenamiento], y[indice_entrenamiento:]

    # Reformatear los datos para que sean 3D (número de muestras, pasos de tiempo, número de características)
    X_entrenamiento = X_entrenamiento.reshape(-1, num_pasos, 1)
    X_prueba = X_prueba.reshape(-1, num_pasos, 1)
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(num_pasos, 1)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Entrenar el modelo
    model.fit(X_entrenamiento, y_entrenamiento, epochs=100, batch_size=32, verbose=1)
    # Evaluar el modelo en los datos de prueba
    loss = model.evaluate(X_prueba, y_prueba)

    # Realizar predicciones
    predicciones = model.predict(X_prueba)
    predicciones_originales = scaler.inverse_transform(predicciones)
    
    # Invertir la normalización en los valores reales de prueba
    y_prueba_originales = scaler.inverse_transform(y_prueba.reshape(-1, 1))

    # Obtener las fechas correspondientes a los datos de prueba
    fechas_prueba = df_datos['Fecha'].values[-len(y_prueba_originales):]

    # Plotear las fechas, valores reales y predicciones
    plt.figure(figsize=(12, 6))
    plt.plot(fechas_prueba, y_prueba_originales, label='Valores reales', marker='o')
    plt.plot(fechas_prueba, predicciones_originales, label='Predicciones', marker='x')
    plt.legend()
    plt.xlabel('Fecha')
    plt.ylabel('Valor')
    plt.title('Predicciones vs. Valores reales con Fechas')
    plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para una mejor visualización
    plt.show()
    
    return predicciones_originales